Includes

In [ ]:
# import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
# import statsmodels.api as sm
# import seaborn as sns
# from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


import cv2
from random import randint
# from PIL import Image
from os import listdir

Compress images

In [ ]:
# def compress_image(path : str):
# 	im = Image.open(path)
# 	im.thumbnail((500, 500), Image.LANCZOS)
# 	im.save(path.replace('dataset', 'reduced_dataset'))

# def compress_dataset(path : str):
# 	for num in range(0,6):
# 		men_dir = path + f'men/{num}/'
# 		women_dir = path + f'Women/{num}/'
# 		for file in listdir(men_dir):
# 			compress_image(men_dir + file)
# 		for file in listdir(women_dir):
# 			compress_image(women_dir + file)

# compress_dataset('../../dataset/')

Read training dataset

In [ ]:
def read_image(path: str) -> np.array:
	img = cv2.imread(path)
	img = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
	img_masked_shadow = np.bitwise_not(cv2.inRange(img,(0,0,80),(255,55,179)))
	img_masked_hand = cv2.inRange(img,(5,35,30),(25,255,255))
	img_filtered = np.bitwise_and(img_masked_shadow,img_masked_hand)
	return img_filtered.reshape(-1)

def read_images(directory : str) -> np.array:
	array = np.array([], dtype=np.int8).reshape(0, 140501)
	for num in range(0,6):
		men_dir = directory + f'men/{num}/'
		women_dir = directory + f'Women/{num}/'
		new_men = np.array([np.concatenate((np.asarray([num]), read_image(men_dir + file)), axis=0) for file in listdir(men_dir)])
		new_women = np.array([np.concatenate((np.asarray([num]), read_image(women_dir + file)), axis=0) for file in listdir(women_dir)])
		array = np.concatenate((array, new_men, new_women), axis=0)
	return array

dataset = read_images('../../reduced_dataset/')
# read_image('../../reduced_dataset/men/0/0_men (20).JPG').shape
# read_image('../../reduced_dataset/men/0/0_men (1).JPG')

In [ ]:
train, validation = train_test_split(dataset, test_size=0.1)
X = train[:, 1:]
y = train[:, 0]
X_val = validation[:, 1:]
y_val = validation[:, 0]

John's Code

In [ ]:
def model_prediction(model, x_training, y_training, x_validation, y_validation):
	model.fit(x_training, y_training)
	
	prediction_training = model.predict(x_training)
	prediction_validation = model.predict(x_validation)
	
	acc_training = accuracy_score(y_training, prediction_training)
	acc_validation = accuracy_score(y_validation, prediction_validation)
	
	print("Accuracuy Score Training: ", acc_training)
	print("Accuracy Score Validation: ",acc_validation)

PCA

In [ ]:
pca = PCA(n_components=36).fit(X)

In [ ]:
print('Random forest')
model_prediction(RandomForestClassifier(), pca.transform(X), y, pca.transform(X_val), y_val)
print('KNN')
model_prediction(KNeighborsClassifier(n_neighbors=3), pca.transform(X), y, pca.transform(X_val), y_val)

In [ ]:
for i in range(10, 50, 2):
	print('*'* 20)
	print(f'PCA {i} components')
	pca = PCA(n_components=i).fit(X)
	print('Random forest')
	model_prediction(RandomForestClassifier(n_estimators = 500, min_samples_split=10, max_features=None), pca.transform(X), y, pca.transform(X_val), y_val)
	# print('KNN, N=5')
	# model_prediction(KNeighborsClassifier(n_neighbors=5), pca.transform(X), y, pca.transform(X_val), y_val)
	print('*'* 20)
	print('\n\n')